In [1]:
run = '02321c49'

In [2]:
from functools import partial
from pathlib import Path
import hydra
from hydra import initialize, compose
from typing import Dict, List
import os

import omegaconf
import pytorch_lightning
from nn_core.common import PROJECT_ROOT
import dataclasses
from typing import List
from nn_core.callbacks import NNTemplateCore
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO
from datasets import Dataset, DatasetDict
import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from torch.utils.data import DataLoader
from pytorch_lightning import Callback
from torch.utils.data import DataLoader
from tqdm import tqdm
import dataclasses
from typing import Union
import random
from hydra.utils import instantiate
from pydoc import locate
from nn_core.serialization import load_model
from typing import List
from nn_core.callbacks import NNTemplateCore
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning import Callback
import logging
import json
import re 

pylogger = logging.getLogger(__name__)

/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/ccmm/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# load 100 epochs 

path = f'../storage/cycle-consistent-model-merging/{run}/checkpoints/'

In [4]:
import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="mlp")

hydra.initialize()

In [5]:
cfg = compose(config_name="mlp", overrides=[])

In [6]:
print(cfg)

{'nn': {'dataset': {'train': {'_target_': 'torchvision.datasets.MNIST', 'root': '${nn.data_path}', 'train': True, 'download': True, 'transform': '${nn.dataset.train_transform}'}, 'test': {'_target_': 'torchvision.datasets.MNIST', 'root': '${nn.data_path}', 'train': False, 'download': True, 'transform': '${nn.dataset.test_transform}'}, 'train_transform': {'_target_': 'torchvision.transforms.Compose', 'transforms': [{'_target_': 'torchvision.transforms.ToTensor'}, {'_target_': 'torchvision.transforms.Normalize', 'mean': [0.1307], 'std': [0.3081]}]}, 'test_transform': '${nn.dataset.train_transform}'}, 'data_path': '${oc.env:PROJECT_ROOT}/data', 'output_path': '${oc.env:PROJECT_ROOT}/output', 'dataset_name': 'MNIST', 'data': {'_target_': 'ccmm.data.datamodule.MyDataModule', 'dataset': '${nn.dataset}', 'gpus': '${train.trainer.gpus}', 'num_workers': {'train': 8, 'val': 4, 'test': 4}, 'batch_size': {'train': 512, 'val': 512, 'test': 512}}, 'module': {'_target_': 'ccmm.pl_modules.pl_module.My

In [7]:
def load_model_ckpt(model_path: str) -> pl.LightningModule:

    model_class = locate("ccmm.pl_modules.pl_module.MyLightningModule")
    model = load_model(model_class, checkpoint_path=Path(model_path))
    model.eval().cuda()
    return model


In [8]:
def load_and_sort_checkpoints(checkpoint_dir):
    # Get all files in the directory
    checkpoint_files = os.listdir(checkpoint_dir)

    # Filter only .ckpt files
    checkpoint_files = [file for file in checkpoint_files if file.endswith('.ckpt.zip')]

    # Sort files based on epoch number
    checkpoint_files.sort(key=lambda x: int(re.search(r'epoch=(\d+)', x).group(1)))

    
    checkpoints = [load_model_ckpt(os.path.join(checkpoint_dir, file)) for file in checkpoint_files]
    return checkpoints

In [9]:
checkpoints = load_and_sort_checkpoints(path)

/data/cycle-consistent-model-merging/src/ccmm/pl_modules/pl_module.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")


In [10]:
models_by_epoch = checkpoints

In [11]:
import torch

K = 10

all_params = []
for epoch_model in models_by_epoch[:K]:
    params = epoch_model.state_dict()
    # flatten and concatenate all the parameters in a single vector
    params = torch.cat([p.flatten() for p in params.values()])
    all_params.append(params)

all_params = torch.stack(all_params)   

In [12]:
# (K, num_params_per_model)
all_params.shape

torch.Size([10, 1061130])

In [13]:
all_differences = []
for i in range(1, K):
    all_differences.append(all_params[i] - all_params[i-1])

all_differences = torch.stack(all_differences)

In [14]:
# shape (num_params_per_model, K-1)
all_differences = all_differences.transpose(1, 0)
all_differences.shape

torch.Size([1061130, 9])

In [15]:
Q, R = torch.linalg.qr(all_differences)

In [16]:
R.shape[0]

9

In [17]:
d = torch.linalg.solve(R.conj().t() @ R, -torch.ones_like(R[0]))
gamma = d / d.sum()

In [18]:
proj_weights = (all_params[:-1].t() * gamma).sum(1)

In [24]:
# copy model_by_epoch[0] to new_model
import copy
new_model = copy.deepcopy(models_by_epoch[0])
# get the original shape of the parameter layers
shapes = [p.shape for p in models_by_epoch[0].state_dict().values()]
# reshape proj_weights into the original shape
proj_weights = proj_weights.split([p.numel() for p in models_by_epoch[0].state_dict().values()])
proj_weights = [p.reshape(s) for p, s in zip(proj_weights, shapes)]
# assign the weights to the model
for p, w in zip(new_model.state_dict().values(), proj_weights):
    p.copy_(w)


AttributeError: 'list' object has no attribute 'split'

In [ ]:
# check the performances of the model on the test set
from ccmm.pl_modules.pl_module import MyLightningModule
from ccmm.pl_modules.data_module import MyDataModule

